# Xception

In [50]:
import densenet_fast

from keras.optimizers import RMSprop, SGD
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.models import model_from_json
from keras.callbacks import TensorBoard

In [2]:
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/'
train_data_dir = DATA_HOME_DIR+'train'

In [3]:
import numpy as np
np.random.seed(2016)

import os
import glob
import cv2
import math
import pickle
import datetime
import pandas as pd

from sklearn.cross_validation import train_test_split
from keras.utils import np_utils
from keras.models import model_from_json
from sklearn.metrics import log_loss

/home/paperspace/anaconda2/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
def split_validation_set(train, target, test_size):
    random_state = 51
    X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [5]:
def split_validation_set_with_hold_out(train, target, test_size):
    random_state = 51
    train, X_test, target, y_test = train_test_split(train, target, test_size=test_size, random_state=random_state)
    X_train, X_holdout, y_train, y_holdout = train_test_split(train, target, test_size=test_size, random_state=random_state)
    return X_train, X_test, X_holdout, y_train, y_test, y_holdout

# Loading the data

In [6]:
from PIL import ImageFilter, ImageStat, Image, ImageDraw
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import glob
import cv2

In [7]:
def im_multi(path):
    try:
        im_stats_im_ = Image.open(path)
        return [path, {'size': im_stats_im_.size}]
    except:
        print(path)
        return [path, {'size': [0,0]}]

def im_stats(im_stats_df):
    im_stats_d = {}
    p = Pool(cpu_count())
    ret = p.map(im_multi, im_stats_df['path'])
    for i in range(len(ret)):
        im_stats_d[ret[i][0]] = ret[i][1]
    im_stats_df['size'] = im_stats_df['path'].map(lambda x: ' '.join(str(s) for s in im_stats_d[x]['size']))
    return im_stats_df

def get_im_cv2(path):
    img = cv2.imread(path)
    resized = cv2.resize(img, (32, 32), cv2.INTER_LINEAR) #use cv2.resize(img, (64, 64), cv2.INTER_LINEAR)
    return [path, resized]

InceptionV3 hat pre_process input: https://github.com/fchollet/deep-learning-models/blob/master/inception_v3.py
def preprocess_input(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

In [8]:
def normalize_image_features(paths):
    imf_d = {}
    p = Pool(cpu_count())
    ret = p.map(get_im_cv2, paths)
    for i in range(len(ret)):
        imf_d[ret[i][0]] = ret[i][1]
    ret = []
    fdata = [imf_d[f] for f in paths]
    fdata = np.array(fdata, dtype=np.uint8)
    fdata = fdata.astype('float32')
    return fdata

In [9]:
train = glob.glob('data/train/**/*.jpg') + glob.glob('data/additional/**/*.jpg')

In [10]:
train[1]

'data/train/Type_3/1127.jpg'

In [11]:
train[1].split('/')

['data', 'train', 'Type_3', '1127.jpg']

In [12]:
train = pd.DataFrame([[p.split('/')[2],p.split('/')[3],p] for p in train], columns = ['type','image','path'])#[::5] #limit for Kaggle Demo

In [13]:
train = im_stats(train)
bad_images = train[train['size'] == '0 0']
train = train[train['size'] != '0 0'].reset_index(drop=True) #remove bad images
print('dropped bad images')
train_data = normalize_image_features(train['path'])
np.save('train.npy', train_data, allow_pickle=True, fix_imports=True)
print('Finished loading data!')

data/additional/Type_2/2845.jpg
data/additional/Type_2/5892.jpg
data/additional/Type_1/5893.jpg
dropped bad images
Finished loading data!


In [14]:
print(bad_images)

        type     image                             path size
5674  Type_2  2845.jpg  data/additional/Type_2/2845.jpg  0 0
6119  Type_2  5892.jpg  data/additional/Type_2/5892.jpg  0 0
7474  Type_1  5893.jpg  data/additional/Type_1/5893.jpg  0 0


In [15]:
le = LabelEncoder()
train_target = le.fit_transform(train['type'].values)
train_target = np_utils.to_categorical(train_target, num_classes=3)

In [16]:
print(train_target[:5])

[[ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]
 [ 0.  0.  1.]]


In [17]:
print(le.classes_) #in case not 1 to 3 order
np.save('train_target_add.npy', train_target, allow_pickle=True, fix_imports=True)

['Type_1' 'Type_2' 'Type_3']


Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (1480, 3, 32, 32) (32 channels).

In [18]:
print(train_data.shape)

(8211, 32, 32, 3)


## Pre-processing

In [24]:
# load data
train_data = np.load('train.npy')
train_target = np.load('train_target_add.npy')

### Zero Centering

In [25]:
mean = np.mean(train_data, axis = 0)

In [26]:
train_data -= np.mean(train_data, axis = 0) # zero-center the data

### Devide by std

In [27]:
std = np.std(train_data, axis = 0)

In [28]:
train_data /= np.std(train_data, axis = 0)

In [30]:
print(train_data[:1])

[[[[-0.57978743 -0.67437583 -0.63709021]
   [-0.63648289 -0.72743857 -0.71886504]
   [-0.67970455 -0.79238492 -0.74779516]
   ..., 
   [-0.62826246 -0.74105406 -0.69575393]
   [-0.62715232 -0.70799261 -0.69360584]
   [-0.59142506 -0.69648713 -0.65484303]]

  [[-0.5506261  -0.64775407 -0.60415184]
   [-0.67455614 -0.78389919 -0.74378467]
   [-0.67878437 -0.78434116 -0.74573082]
   ..., 
   [-0.74030143 -0.80144435 -0.7648918 ]
   [-0.65657914 -0.77183163 -0.72525138]
   [-0.65881282 -0.77619058 -0.72624516]]

  [[-0.69404757 -0.79054666 -0.77565485]
   [-0.72249603 -0.83344954 -0.7999841 ]
   [-0.73673767 -0.84566861 -0.799541  ]
   ..., 
   [-0.62531322 -0.71582103 -0.71077734]
   [-0.61915886 -0.75715697 -0.71709776]
   [-0.68576413 -0.80234766 -0.75442439]]

  ..., 
  [[-0.65991127 -0.73984611 -0.75051588]
   [-0.65508115 -0.7647934  -0.76209873]
   [-0.64980668 -0.72460431 -0.75910383]
   ..., 
   [-0.60023999 -0.69549078 -0.71221095]
   [-0.58739507 -0.71632898 -0.71207792]
   [-0.

### Scale pixel values to [0,1]

In [31]:
train_data = train_data / 255

In [32]:
print(train_data[:1])

[[[[-0.00227368 -0.00264461 -0.00249839]
   [-0.00249601 -0.0028527  -0.00281908]
   [-0.00266551 -0.00310739 -0.00293253]
   ..., 
   [-0.00246377 -0.00290609 -0.00272845]
   [-0.00245942 -0.00277644 -0.00272002]
   [-0.00231931 -0.00273132 -0.00256801]]

  [[-0.00215932 -0.00254021 -0.00236922]
   [-0.00264532 -0.00307411 -0.0029168 ]
   [-0.0026619  -0.00307585 -0.00292443]
   ..., 
   [-0.00290314 -0.00314292 -0.00299958]
   [-0.00257482 -0.00302679 -0.00284412]
   [-0.00258358 -0.00304388 -0.00284802]]

  [[-0.00272176 -0.00310018 -0.00304178]
   [-0.00283332 -0.00326843 -0.00313719]
   [-0.00288917 -0.00331635 -0.00313545]
   ..., 
   [-0.00245221 -0.00280714 -0.00278736]
   [-0.00242807 -0.00296924 -0.00281215]
   [-0.00268927 -0.00314646 -0.00295853]]

  ..., 
  [[-0.00258789 -0.00290136 -0.0029432 ]
   [-0.00256895 -0.00299919 -0.00298862]
   [-0.00254826 -0.00284159 -0.00297688]
   ..., 
   [-0.00235388 -0.00272741 -0.00279298]
   [-0.00230351 -0.00280913 -0.00279246]
   [-0.

## Splitting the data

In [33]:
X_train, X_val, Y_train, Y_val = split_validation_set(train_data, train_target, 0.3)

In [34]:
print(X_train.shape)

(5747, 32, 32, 3)


# Now building the model

In [35]:
def printTrainableLayers(model):
    for i, layer in enumerate(model.layers):
       if layer.trainable == True: 
           print(i, layer.name)

In [36]:
def printAllLayers(model):
    for i, layer in enumerate(model.layers):
       print(i, layer.name)

In [37]:
image_dim = (32, 32, 3)

In [46]:
model = densenet_fast.create_dense_net(nb_classes=3, img_dim=image_dim, depth=40, growth_rate=12, dropout_rate=0.2, 
                                       verbose=True)

DenseNet-40-12 created.


In [47]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 32, 32, 3)     0                                            
____________________________________________________________________________________________________
initial_conv2D (Conv2D)          (None, 32, 32, 16)    432         input_2[0][0]                    
____________________________________________________________________________________________________
batch_normalization_4 (BatchNorm (None, 32, 32, 16)    64          initial_conv2D[0][0]             
____________________________________________________________________________________________________
activation_38 (Activation)       (None, 32, 32, 16)    0           batch_normalization_4[0][0]      
___________________________________________________________________________________________

In [48]:
filepath="densenet_fast-improve-{epoch:02d}-{val_loss:}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
tbCallBack = TensorBoard(log_dir='./graph_densenet', histogram_freq=0, write_graph=True, write_images=True)
callbacks_list.append(tbCallBack)

In [51]:
#opt = RMSprop(lr=1e-2, rho=0.9, decay=0.01)
opt = SGD(lr=0.1, momentum=0.9, decay=0.9, nesterov=True)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [52]:
batch_size = 16
nb_classes = 3
nb_epoch = 150
nb_train_samples = X_train.shape[0]
nb_val_samples = X_val.shape[0]

In [53]:
#rescale=1./255, removed because of pre_process
datagen = ImageDataGenerator()
datagen.fit(train_data)

In [54]:
train_generator = datagen.flow(X_train,Y_train)

Sample hat shape: (32,32,32,3)

In [ ]:
history = model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=nb_epoch,
        validation_data=(X_val, Y_val),
        validation_steps=nb_val_samples // batch_size,
        callbacks=callbacks_list)
print('Finished!')

Epoch 1/150
359/359 [==============================] - 61s - loss: 1.2198 - categorical_accuracy: 0.5344 - val_loss: 1.2853 - val_categorical_accuracy: 0.5041
Epoch 2/150
359/359 [==============================] - 54s - loss: 1.2046 - categorical_accuracy: 0.5393 - val_loss: 1.2345 - val_categorical_accuracy: 0.5077
Epoch 3/150
359/359 [==============================] - 54s - loss: 1.1984 - categorical_accuracy: 0.5398 - val_loss: 1.2260 - val_categorical_accuracy: 0.5069
Epoch 4/150
359/359 [==============================] - 54s - loss: 1.2006 - categorical_accuracy: 0.5404 - val_loss: 1.2261 - val_categorical_accuracy: 0.5045
Epoch 5/150
359/359 [==============================] - 54s - loss: 1.1987 - categorical_accuracy: 0.5404 - val_loss: 1.2250 - val_categorical_accuracy: 0.5045
Epoch 6/150
359/359 [==============================] - 54s - loss: 1.1999 - categorical_accuracy: 0.5395 - val_loss: 1.2253 - val_categorical_accuracy: 0.5045
Epoch 7/150
359/359 [=========================

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("xcep_top_only.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("xcep_top_only.h5")
print("Saved model to disk")

# Submit to kaggle for testing purposes

## Loading test data

In [ ]:
test = glob.glob('data/test/**/*.jpg')
test = pd.DataFrame([[p.split('/')[3],p] for p in test], columns = ['image','path']) #[::20] #limit for Kaggle Demo
test_data = normalize_image_features(test['path'])
np.save('test.npy', test_data, allow_pickle=True, fix_imports=True)

test_id = test.image.values
np.save('test_id.npy', test_id, allow_pickle=True, fix_imports=True)

In [ ]:
print(test_data.shape)

In [ ]:
test_data_channels_last = np.transpose(test_data, (0, 2, 3, 1))

In [ ]:
print(test_data_channels_last.shape)

In [ ]:
# Calculate class posteriors probabilities
y_probabilities = model.predict(test_data_channels_last, batch_size=16, verbose=0)
print(len(y_probabilities))

In [ ]:
print(y_probabilities[:5])

In [ ]:
df = pd.DataFrame(y_probabilities, columns=['Type_1','Type_2','Type_3'])
df['image_name'] = test_id
df.to_csv('submission.csv', index=False)

In [ ]:
%ls